### Set up functions and routes

In [1]:
def get_time(location: str) -> str:
    """Useful to get the time in a specific location"""
    print(f"Calling `get_time` function with location: {location}")
    return "get_time"


def get_news(category: str, country: str) -> str:
    """Useful to get the news in a specific country"""
    print(
        f"Calling `get_news` function with category: {category} and country: {country}"
    )
    return "get_news"

Now generate a dynamic routing config for each function

In [2]:
from semantic_router import Route, RouteConfig

functions = [get_time, get_news]
routes = []

for function in functions:
    route = await Route.from_dynamic_route(entity=function)
    routes.append(route)

route_config = RouteConfig(routes=routes)

/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
2023-12-19 12:30:53 INFO semantic_router.utils.logger Generating dynamic route...
2023-12-19 12:30:58 INFO semantic_router.utils.logger Generated route config:
{
    "name": "get_time",
    "utterances": [
        "What's the time in [location]?",
        "Can you tell me the time in [location]?",
        "I need to know the time in [location].",
        "What time is it in [location]?",
        "Can you give me the time in [location]?"
    ]
}
2023-12-19 12:30:58 INFO semantic_router.utils.logger Generating dynamic route...
2023-12-19 12:31:03 INF

In [3]:
# You can manually add or remove routes

get_weather_route = Route(
    name="get_weather",
    utterances=[
        "what is the weather in SF",
        "what is the current temperature in London?",
        "tomorrow's weather in Paris?",
    ],
)
route_config.add(get_weather_route)

route_config.remove("get_time")

route_config.to_dict()

2023-12-19 12:31:03 INFO semantic_router.utils.logger Added route `get_weather`
2023-12-19 12:31:03 INFO semantic_router.utils.logger Removed route `get_time`


[{'name': 'get_news',
  'utterances': ['Tell me the latest news from the US',
   "What's happening in India today?",
   'Get me the top stories from Japan',
   'Can you give me the breaking news from Brazil?',
   "What's the latest in Germany?"],
  'description': None},
 {'name': 'get_weather',
  'utterances': ['what is the weather in SF',
   'what is the current temperature in London?',
   "tomorrow's weather in Paris?"],
  'description': None}]

In [4]:
# Get a route by name
route_config.get("get_weather")

Route(name='get_weather', utterances=['what is the weather in SF', 'what is the current temperature in London?', "tomorrow's weather in Paris?"], description=None)

Save config to a file (.json or .yaml)

In [5]:
route_config.to_file("route_config.json")

2023-12-19 12:31:50 INFO semantic_router.utils.logger Saving route config to route_config.json


Load from local file

In [6]:
route_config = RouteConfig.from_file("route_config.json")

2023-12-19 12:32:24 INFO semantic_router.utils.logger Loading route config from route_config.json


### Define routing layer

In [7]:
from semantic_router import RouteLayer

route_layer = RouteLayer(routes=route_config.routes)

In [ ]:
def validate_parameters(function, parameters):
    sig = inspect.signature(function)
    for name, param in sig.parameters.items():
        if name not in parameters:
            return False, f"Parameter {name} missing from query"
        if not isinstance(parameters[name], param.annotation):
            return False, f"Parameter {name} is not of type {param.annotation}"
    return True, "Parameters are valid"

In [ ]:
def extract_parameters(query: str, function) -> dict:
    logger.info("Extracting parameters...")
    example_query = "How is the weather in Hawaii right now in International units?"

    example_schema = {
        "name": "get_weather",
        "description": "Useful to get the weather in a specific location",
        "signature": "(location: str, degree: str) -> str",
        "output": "<class 'str'>",
    }

    example_parameters = {
        "location": "London",
        "degree": "Celsius",
    }

    prompt = f"""
    You are a helpful assistant designed to output JSON.
    Given the following function schema
    << {get_function_schema(function)} >>
    and query
    << {query} >>
    extract the parameters values from the query, in a valid JSON format.
    Example:
    Input:
    query: {example_query}
    schema: {example_schema}

    Result: {example_parameters}

    Input:
    query: {query}
    schema: {get_function_schema(function)}
    Result:
    """

    try:
        ai_message = llm_mistral(prompt)
        ai_message = (
            ai_message.replace("Output:", "").replace("'", '"').strip().rstrip(",")
        )
    except Exception as e:
        logger.error(f"Mistral failed with error {e}, falling back to OpenAI")
        ai_message = llm_openai(prompt)

    try:
        parameters = json.loads(ai_message)
        valid, message = validate_parameters(function, parameters)

        if not valid:
            logger.warning(
                f"Invalid parameters from Mistral, falling back to OpenAI: {message}"
            )
            # Fall back to OpenAI
            ai_message = llm_openai(prompt)
            parameters = json.loads(ai_message)
            valid, message = validate_parameters(function, parameters)
            if not valid:
                raise ValueError(message)

        logger.info(f"Extracted parameters: {parameters}")
        return parameters
    except ValueError as e:
        logger.error(f"Parameter validation error: {str(e)}")
        return {"error": "Failed to validate parameters"}

Set up calling functions

In [ ]:
from typing import Callable
from semantic_router.layer import RouteLayer


def call_function(function: Callable, parameters: dict[str, str]):
    try:
        return function(**parameters)
    except TypeError as e:
        logger.error(f"Error calling function: {e}")


def call_llm(query: str) -> str:
    try:
        ai_message = llm_mistral(query)
    except Exception as e:
        logger.error(f"Mistral failed with error {e}, falling back to OpenAI")
        ai_message = llm_openai(query)

    return ai_message


def call(query: str, functions: list[Callable], router: RouteLayer):
    function_name = router(query)
    if not function_name:
        logger.warning("No function found")
        return call_llm(query)

    for function in functions:
        if function.__name__ == function_name:
            parameters = extract_parameters(query, function)
            print(f"parameters: {parameters}")
            return call_function(function, parameters)

### Workflow

Functions as a tool

In [ ]:
# Loading configuration from file
router = RouteLayer.from_json("router.json")

In [ ]:
tools = [get_time, get_news]

call(query="What is the time in Stockholm?", functions=tools, router=router)
call(query="What is the tech news in the Lithuania?", functions=tools, router=router)
call(query="Hi!", functions=tools, router=router)